In [34]:
# --- Импорт Необходимых Библиотек ---
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point, LineString
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.spatial import cKDTree
from shapely.ops import unary_union
import time
import numpy as np
import random
import seaborn as sns
from tqdm import tqdm
from multiprocessing.pool import ThreadPool
from multiprocessing import cpu_count

# --- Настройки ---
# Путь к директории с данными
data_dir = "C:\\Users\\hedge\\DataspellProjects\\cp_2024_graphs\\Векторные данные\\"

# Путь к директории для сохранения версий графа
versions_dir = os.path.join(data_dir, "Versions")
os.makedirs(versions_dir, exist_ok=True)

# Целевая система координат
target_crs = "EPSG:3857"

# Пропускная способность пешеходной дорожки (чел/ч)
P = 800

# --- Функции ---

def load_and_reproject(file_path, target_crs="EPSG:3857"):
    """
    Загружает GeoDataFrame из файла и приводит его к целевой CRS.
    """
    try:
        gdf = gpd.read_file(file_path)
        gdf = gdf.to_crs(target_crs)
        print(f"Файл {file_path} загружен успешно с {len(gdf)} записями.")
        return gdf
    except Exception as e:
        print(f"Ошибка при загрузке файла {file_path}: {e}")
        return None

def fix_winding_order(geom):
    """
    Исправляет порядок обхода колец полигона.
    Внешнее кольцо должно обходиться по часовой стрелке (CW),
    внутренние (дыры) — против часовой стрелки (CCW).
    """
    if geom is None:
        return geom

    if isinstance(geom, Polygon):
        exterior = geom.exterior
        interiors = list(geom.interiors)

        # Обеспечиваем, что внешний контур обходится по часовой стрелке
        if exterior.is_ccw:
            exterior = Polygon(exterior.coords[::-1]).exterior

        # Внутренние кольца (дыры) должны обходиться против часовой стрелки
        fixed_interiors = []
        for interior in interiors:
            if not interior.is_ccw:
                fixed_interiors.append(interior.coords[::-1])
            else:
                fixed_interiors.append(interior.coords)

        return Polygon(exterior, fixed_interiors)

    elif isinstance(geom, MultiPolygon):
        fixed_polygons = [fix_winding_order(p) for p in geom.geoms]
        return MultiPolygon(fixed_polygons)

    else:
        return geom

def safe_fix_winding_order(geom):
    """
    Безопасно применяет функцию исправления порядка обхода.
    В случае ошибки возвращает исходную геометрию.
    """
    try:
        return fix_winding_order(geom)
    except Exception as e:
        print(f"Ошибка при обработке геометрии: {e}")
        return geom

def further_fix_geometry(gdf):
    """
    Дополнительное исправление геометрий с помощью buffer(0).
    """
    gdf['geometry'] = gdf['geometry'].buffer(0)
    return gdf

def clean_geometries(gdf):
    """
    Исправляет порядок обхода колец и корректирует геометрию.
    """
    gdf['geometry'] = gdf['geometry'].apply(safe_fix_winding_order)
    invalid_geometries = gdf[~gdf.is_valid]
    print(f"Количество некорректных геометрий после исправления: {len(invalid_geometries)}")

    if not invalid_geometries.empty:
        print("Некорректные геометрии найдены:")
        print(invalid_geometries[['Name', 'Type', 'Purpose', 'geometry']].head())
        # Применение buffer(0) для дополнительного исправления
        gdf = further_fix_geometry(gdf)
        # Повторная проверка
        invalid_geometries = gdf[~gdf.is_valid]
        print(f"Количество некорректных геометрий после дополнительного исправления: {len(invalid_geometries)}")
        if not invalid_geometries.empty:
            print("Некорректные геометрии после дополнительного исправления:")
            print(invalid_geometries[['Name', 'Type', 'Purpose', 'geometry']].head())
        else:
            print("Все геометрии корректны после дополнительного исправления.")
    else:
        print("Все геометрии корректны.")

    return gdf

def build_graph(gdf):
    G = nx.Graph()

    for idx, row in gdf.iterrows():
        geometry = row.geometry
        if geometry is None:
            continue

        # Предполагаем, что геометрия - это LineString или MultiLineString
        if geometry.type == 'LineString':
            lines = [geometry]
        elif geometry.type == 'MultiLineString':
            lines = list(geometry)
        else:
            continue  # Пропускаем геометрии, не являющиеся линиями

        for line in lines:
            coords = list(line.coords)
            for i in range(len(coords) - 1):
                start = coords[i]
                end = coords[i + 1]

                # Добавляем узлы с атрибутами координат
                if not G.has_node(start):
                    G.add_node(start, pos=start)
                if not G.has_node(end):
                    G.add_node(end, pos=end)

                # Добавляем ребро с атрибутами из строки таблицы
                edge_attrs = row.to_dict()
                G.add_edge(start, end, **edge_attrs)

    return G

def get_largest_component(G):
    """
    Возвращает подграф самого большого связного компонента.
    """
    connected_components = list(nx.connected_components(G))
    connected_components = sorted(connected_components, key=len, reverse=True)
    print(f"Количество связанных компонентов в графе: {len(connected_components)}")

    for i, component in enumerate(connected_components, 1):
        print(f"Компонент {i}: {len(component)} узлов")

    largest_component = connected_components[0]
    print(f"Размер самого большого компонента: {len(largest_component)} узлов")

    G_largest = G.subgraph(largest_component).copy()
    print(f"Граф с самым большим компонентом имеет {G_largest.number_of_nodes()} узлов и {G_largest.number_of_edges()} ребер.")

    if nx.is_connected(G_largest):
        print("Граф с самым большим компонентом является связным.")
    else:
        print("Граф с самым большим компонентом не является полностью связным.")

    return G_largest

def find_closest_edge(G, node):
    """
    Находит ближайшее ребро графа к заданному узлу.
    
    Parameters:
    - G: NetworkX граф
    - node: координаты узла (tuple)
    
    Returns:
    - closest_edge: tuple (u, v) ближайшего ребра или None
    """
    min_distance = float('inf')
    closest_edge = None
    for edge in G.edges():
        u, v = edge
        line = LineString([u, v])
        point = Point(node)
        distance = line.distance(point)
        if distance < min_distance:
            min_distance = distance
            closest_edge = edge
    return closest_edge if closest_edge else None

def bind_objects_to_graph(G, objects_gdf, object_type="Object"):
    """
    Привязывает объекты (дома или ключевые точки) к ближайшим узлам графа с использованием cKDTree.
    Фильтрует объекты с пустой геометрией.
    
    Parameters:
    - G: NetworkX граф
    - objects_gdf: GeoDataFrame с объектами
    - object_type: str, тип объекта для логирования
    """
    # Фильтрация объектов с пустой геометрией
    initial_count = len(objects_gdf)
    objects_gdf = objects_gdf[~objects_gdf.geometry.is_empty]
    filtered_count = len(objects_gdf)
    removed_count = initial_count - filtered_count
    if removed_count > 0:
        print(f"Удалено {removed_count} {object_type}(ов) с пустой геометрией.")

    # Если после фильтрации нет объектов, прекращаем обработку
    if objects_gdf.empty:
        print(f"Нет объектов типа '{object_type}' для привязки.")
        return objects_gdf

    nodes = list(G.nodes)
    node_coords = np.array(nodes)
    tree = cKDTree(node_coords)

    # Извлечение координат центроидов объектов
    centroids = objects_gdf.geometry.centroid
    coords = []
    empty_centroid_indices = []
    for idx, point in centroids.items():  # Заменено iteritems() на items()
        if point.is_empty:
            empty_centroid_indices.append(idx)
            coords.append((float('nan'), float('nan')))
        else:
            coords.append((point.x, point.y))

    coords = np.array(coords)

    # Удаление объектов с пустыми центроидами
    if empty_centroid_indices:
        print(f"Найдено {len(empty_centroid_indices)} {object_type}(ов) с пустыми центроидами. Они будут удалены.")
        objects_gdf = objects_gdf.drop(index=empty_centroid_indices)
        coords = np.delete(coords, empty_centroid_indices, axis=0)

    # Если после удаления пустых центроидов нет объектов, прекращаем обработку
    if objects_gdf.empty:
        print(f"Нет объектов типа '{object_type}' с валидными центроидами для привязки.")
        return objects_gdf

    # Поиск ближайших узлов
    distances, indices = tree.query(coords, k=1)

    # Присвоение ближайших узлов
    objects_gdf.loc[:, 'nearest_node'] = [nodes[idx] for idx in indices]  # Использовано .loc

    # Проверка наличия None (в данном случае не должно быть, так как KDTree всегда находит ближайшего)
    missing_nodes = objects_gdf['nearest_node'].isna().sum()
    print(f"\nКоличество {object_type}(ов) без привязки к узлам графа: {missing_nodes}")

    if missing_nodes > 0:
        print(f"Некоторые {object_type}(ы) не были привязаны к узлам графа. Рассмотрите возможность удаления или ручной привязки.")
    else:
        print(f"Все {object_type}(ы) успешно привязаны к ближайшим узлам графа.")

    return objects_gdf

def compute_shortest_distances(G, key_nodes):
    """
    Вычисляет кратчайшие расстояния от всех узлов до ближайшего ключевого узла.
    
    Parameters:
    - G: NetworkX граф
    - key_nodes: список узлов ключевых точек
    
    Returns:
    - shortest_distances: словарь с узлами и их расстояниями до ближайшего ключевого узла
    """
    start_time = time.time()
    # Используем multi_source_dijkstra_path_length для эффективного вычисления
    shortest_distances = nx.multi_source_dijkstra_path_length(G, key_nodes, weight='weight')
    elapsed_time = time.time() - start_time
    print(f"Время выполнения многосходного поиска: {elapsed_time:.2f} секунд")
    return shortest_distances

def visualize_accessibility(G, houses_gdf, key_points_gdf, year):
    """
    Визуализирует результаты пешеходной доступности.
    """
    threshold = houses_gdf['accessibility'].quantile(0.75)
    print(f"Порог для низкой доступности (75-й процентиль): {threshold}")

    fig, ax = plt.subplots(figsize=(12, 12))

    # Существующие улицы
    streets_gdf = gpd.GeoDataFrame(geometry=[LineString(edge) for edge in G.edges()], crs=target_crs)
    streets_gdf.plot(ax=ax, color='gray', linewidth=0.5, label='Улицы')

    # Дома с доступностью
    houses_gdf.plot(ax=ax, column='accessibility', cmap='OrRd', markersize=50, legend=True, label='Доступность')

    # Дома с низкой доступностью
    low_access = houses_gdf[houses_gdf['accessibility'] > threshold]
    low_access.plot(ax=ax, color='blue', markersize=50, label='Низкая доступность')

    # Ключевые точки
    key_points_gdf.plot(ax=ax, color='green', markersize=100, marker='*', label='Ключевые точки')

    plt.legend()
    plt.title(f"Анализ Пешеходной Доступности - {year}")
    plt.xlabel("X координата (м)")
    plt.ylabel("Y координата (м)")
    plt.show()

def save_versioned_graph(G, year, output_dir):
    """
    Сохраняет граф G как shapefile с указанием года.
    """
    edges = []
    for u, v, data in G.edges(data=True):
        line = LineString([u, v])
        edges.append({
            'geometry': line,
            'weight': data.get('weight', 1)
        })

    gdf_edges = gpd.GeoDataFrame(edges, crs=target_crs)
    output_path = os.path.join(output_dir, f"Streets_{year}.shp")
    gdf_edges.to_file(output_path)
    print(f"Дорожной граф сохранен как {output_path}")

import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, Point
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
import warnings

# Перемещаем функцию на верхний уровень для корректной сериализации
def process_public_transport_load(args):
    """
    Функция для обработки одной записи дома при агрегации нагрузки от общественного транспорта.

    Parameters:
    - G: NetworkX граф (должен быть глобально доступен)
    - house: dict с данными дома

    Returns:
    - tuple: (список frozenset ребер, нагрузка на ребро)
    """
    G, key_nodes, house = args
    source = house['nearest_node']
    load_public_transport = house['intensity_adults_public_transport']
    try:
        # Поиск ближайшего ключевого узла по кратчайшему пути
        target = min(
            key_nodes,
            key=lambda node: nx.shortest_path_length(G, source=source, target=node, weight='weight')
        )
        # Нахождение кратчайшего пути
        path = nx.shortest_path(G, source=source, target=target, weight='weight')
        num_edges = len(path) - 1
        if num_edges > 0:
            load_per_edge = load_public_transport / num_edges
            # Преобразование пути в список frozenset ребер
            edge_fs_list = [frozenset([path[i], path[i+1]]) for i in range(num_edges)]
            return edge_fs_list, load_per_edge
    except nx.NetworkXNoPath:
        # Нет пути до ключевого узла
        print(f"Нет пути от узла {source} до ключевого узла.")
    except Exception as e:
        print(f"Ошибка при обработке дома с узлом {source}: {e}")
    return [], 0.0

def aggregate_load_on_edges_optimized(G, houses_gdf, key_nodes_gdf, target_crs):
    """
    Оптимизированная функция для агрегации нагрузки на ребра графа.
    
    Parameters:
    - G: NetworkX граф
    - houses_gdf: GeoDataFrame с данными домов, включая 'nearest_node', 
                  'intensity_children_pensioners', 'intensity_adults_public_transport'
    - key_nodes_gdf: GeoDataFrame с узлами ключевых точек (например, остановки ОТ, выходы метро)
    - target_crs: целевая система координат
    
    Returns:
    - edge_loads: dict с ребрами (как frozenset) и их суммарной нагрузкой
    """
    # Игнорировать предупреждения SettingWithCopyWarning
    warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

    # Конвертация ребер графа в GeoDataFrame
    edges = []
    pos = nx.get_node_attributes(G, 'pos')  # Предполагается, что координаты хранятся в 'pos' как (x, y)
    for u, v, data in G.edges(data=True):
        edge_fs = frozenset([u, v])
        try:
            geometry = LineString([pos[u], pos[v]])
            edges.append({'edge_fs': edge_fs, 'geometry': geometry})
        except KeyError as e:
            print(f"Узел {e} не содержит координат. Ребро {u}-{v} будет пропущено.")
            continue

    if not edges:
        print("Список ребер пуст после обработки. Возвращаем пустой словарь нагрузок.")
        return {}

    # Создание GeoDataFrame с явным указанием столбца geometry
    edges_gdf = gpd.GeoDataFrame(edges, crs=target_crs, geometry='geometry')

    # Создание пространственного индекса
    edges_sindex = edges_gdf.sindex

    # Инициализация Series для нагрузок на ребрах
    edge_loads = pd.Series(0.0, index=edges_gdf['edge_fs'])

    # Обработка нагрузок от детей и пенсионеров (load_children)
    houses_with_children = houses_gdf[houses_gdf['intensity_children_pensioners'] > 0].copy()
    if not houses_with_children.empty:
        # Извлечение координат узлов домов
        house_nodes = houses_with_children['nearest_node'].tolist()
        pos_nodes = nx.get_node_attributes(G, 'pos')  # Координаты узлов

        # Проверка наличия координат для всех узлов
        missing_nodes = [node for node in house_nodes if node not in pos_nodes]
        if missing_nodes:
            print(f"Узлы {missing_nodes} не содержат координат и будут пропущены.")
            houses_with_children = houses_with_children[~houses_with_children['nearest_node'].isin(missing_nodes)]
            house_nodes = houses_with_children['nearest_node'].tolist()

        if not houses_with_children.empty:
            # Создание GeoSeries точек домов
            house_points = gpd.GeoSeries([Point(pos_nodes[node]) for node in house_nodes], crs=target_crs)

            # Поиск ближайших ребер для всех домов
            nearest_edges = []
            for point in tqdm(house_points, desc="Поиск ближайших ребер для load_children"):
                try:
                    # Использование sindex.nearest с геометрией напрямую (предполагается, что geopandas >= 0.10)
                    nearest_geom_indices = list(edges_sindex.nearest(point, 1))
                except AttributeError:
                    # Для старых версий GeoPandas, использующих R-tree напрямую
                    nearest_geom_indices = list(edges_sindex.nearest(point.bounds, 1))

                if not nearest_geom_indices:
                    nearest_edges.append(None)
                    continue

                # В некоторых версиях nearest может возвращать кортежи (индекс, дистанция)
                if isinstance(nearest_geom_indices[0], (tuple, list)):
                    nearest_geom_index = nearest_geom_indices[0][0]
                else:
                    nearest_geom_index = nearest_geom_indices[0]

                # Проверка корректности индекса
                if not isinstance(nearest_geom_index, int):
                    print(f"Некорректный индекс: {nearest_geom_index}. Пропускаем.")
                    nearest_edges.append(None)
                    continue

                # Извлечение соответствующего ребра
                try:
                    nearest_geom = edges_gdf.iloc[nearest_geom_index]
                    nearest_edges.append(nearest_geom['edge_fs'])
                except IndexError:
                    print(f"Индекс {nearest_geom_index} выходит за пределы диапазона. Пропускаем.")
                    nearest_edges.append(None)

            # Добавление ближайших ребер в DataFrame с использованием .assign для избежания SettingWithCopyWarning
            houses_with_children = houses_with_children.assign(nearest_edge_fs=nearest_edges)

            # Удаление домов, для которых не удалось найти ближайшее ребро
            houses_with_children = houses_with_children.dropna(subset=['nearest_edge_fs'])

            # Группировка и суммирование нагрузок
            load_children_grouped = houses_with_children.groupby('nearest_edge_fs')['intensity_children_pensioners'].sum()

            # Обновление нагрузок на ребрах
            edge_loads = edge_loads.add(load_children_grouped, fill_value=0)

    # # Обработка нагрузок от взрослых, использующих общественный транспорт (load_public_transport)
    # houses_with_public_transport = houses_gdf[houses_gdf['intensity_adults_public_transport'] > 0].copy()
    # key_nodes = key_nodes_gdf['nearest_node'].dropna().unique().tolist()
    # 
    # if not key_nodes:
    #     print("Список ключевых узлов пуст. Нагрузка от взрослых, использующих общественный транспорт, не будет распределяться.")
    # 
    # if not houses_with_public_transport.empty and key_nodes:
    #     # Подготовка данных для multiprocessing
    #     houses_list = houses_with_public_transport.to_dict('records')
    # 
    #     # Параллельная обработка домов с отображением прогресса
    #     # Передаём G и key_nodes как часть аргументов, чтобы избежать проблем с глобальными переменными
    #     args = [(G, key_nodes, house) for house in houses_list]
    # 
    #     with Pool(processes=cpu_count()) as pool:
    #         results = list(tqdm(
    #             pool.imap(process_public_transport_load, args),
    #             total=len(args),
    #             desc="Агрегация нагрузок от public_transport"
    #         ))
    # 
    #     # Объединение результатов
    #     for edge_fs_list, load_per_edge in results:
    #         for edge_fs in edge_fs_list:
    #             if edge_fs in edge_loads:
    #                 edge_loads[edge_fs] += load_per_edge
    #             else:
    #                 # Если ребро не найдено в edges_gdf, пропускаем
    #                 print(f"Ребро {edge_fs} не найдено в графе и будет пропущено.")

    # Преобразование Series в dict и удаление нулевых нагрузок
    edge_loads = edge_loads[edge_loads > 0]
    return edge_loads.to_dict()

from multiprocessing import Pool, cpu_count
def analyze_peak_load(edge_loads, G):
    """
    Анализирует загруженность пешеходных дорожек на основе агрегированных нагрузок.
    
    Parameters:
    - edge_loads: словарь с ребрами (как frozenset) и их нагрузкой
    - G: NetworkX граф
    """
    # Преобразование в GeoDataFrame
    edges = []
    for edge_fs, load in edge_loads.items():
        u, v = tuple(edge_fs)
        line = LineString([u, v])
        edges.append({
            'geometry': line,
            'load': load
        })

    edges_gdf = gpd.GeoDataFrame(edges, crs=target_crs)

    # Идентификация перегруженных участков
    problematic_zones = edges_gdf[edges_gdf['load'] > P]
    print(f"Количество проблемных зон: {len(problematic_zones)}")

    # Визуализация перегруженных зон
    if not problematic_zones.empty:
        fig, ax = plt.subplots(figsize=(12, 12))
        streets_gdf = gpd.GeoDataFrame(geometry=[LineString(edge) for edge in G.edges()], crs=target_crs)
        streets_gdf.plot(ax=ax, color='gray', linewidth=0.5, label='Улицы')
        problematic_zones.plot(ax=ax, color='red', linewidth=2, label='Перегруженные участки')
        plt.legend()
        plt.title("Перегруженные Пешеходные Дорожки")
        plt.xlabel("X координата (м)")
        plt.ylabel("Y координата (м)")
        plt.show()
    else:
        print("Нет перегруженных пешеходных дорожек.")

def plot_distribution(houses_gdf, edge_loads):
    """
    Строит гистограммы распределения интенсивности и нагрузки на ребрах.
    """
    plt.figure(figsize=(14, 6))

    # Распределение интенсивности
    plt.subplot(1, 2, 1)
    sns.histplot(houses_gdf['total_intensity'], bins=50, kde=True, color='blue')
    plt.title('Распределение Интенсивности')
    plt.xlabel('Интенсивность')
    plt.ylabel('Количество домов')

    # Распределение нагрузки на ребрах
    load_values = list(edge_loads.values())
    plt.subplot(1, 2, 2)
    sns.histplot(load_values, bins=50, kde=True, color='red')
    plt.title('Распределение Нагрузки на Ребрах')
    plt.xlabel('Нагрузка (чел/ч)')
    plt.ylabel('Количество ребер')

    plt.tight_layout()
    plt.show()

def save_versioned_graph(G, year, output_dir):
    """
    Сохраняет граф G как shapefile с указанием года.
    """
    edges = []
    for u, v, data in G.edges(data=True):
        line = LineString([u, v])
        edges.append({
            'geometry': line,
            'weight': data.get('weight', 1)
        })

    gdf_edges = gpd.GeoDataFrame(edges, crs=target_crs)
    output_path = os.path.join(output_dir, f"Streets_{year}.shp")
    gdf_edges.to_file(output_path)
    print(f"Дорожной граф сохранен как {output_path}")

def suggest_new_pedestrian_paths_heuristic(G, buildings_gdf, existing_paths_gdf, num_new_paths=10):
    """
    Предлагает новые пешеходные дорожки на основе эвристических правил.
    
    Parameters:
    - G: NetworkX граф
    - buildings_gdf: GeoDataFrame с существующими и планируемыми зданиями
    - existing_paths_gdf: GeoDataFrame с уже существующими пешеходными дорожками
    - num_new_paths: количество предлагаемых новых дорожек
    
    Returns:
    - new_paths_gdf: GeoDataFrame с новыми дорожками или None
    """
    # Объединение всех зданий для упрощения проверки пересечений
    buildings_union = unary_union(buildings_gdf.geometry)

    new_paths = []
    nodes = list(G.nodes)

    attempts = 0
    max_attempts = num_new_paths * 10  # Ограничение количества попыток

    while len(new_paths) < num_new_paths and attempts < max_attempts:
        source, target = random.sample(nodes, 2)
        # Используем кратчайший путь по весу (длине)
        try:
            path = nx.shortest_path(G, source=source, target=target, weight='weight')
            line = LineString(path)
            # Проверка пересечения с зданиями
            if not line.crosses(buildings_union):
                # Проверка, что такая дорожка еще не существует
                if not existing_paths_gdf.intersects(line).any():
                    new_paths.append({
                        'geometry': line,
                        'weight': line.length
                    })
        except nx.NetworkXNoPath:
            pass  # Нет пути между узлами
        attempts += 1

    if new_paths:
        new_paths_gdf = gpd.GeoDataFrame(new_paths, crs=target_crs)
        print(f"Предложено {len(new_paths)} новых пешеходных дорожек.")
        return new_paths_gdf
    else:
        print("Не удалось предложить новые пешеходные дорожки без пересечения с зданиями.")
        return None

def update_graph_with_new_paths(G, new_paths_gdf):
    """
    Обновляет граф G новыми пешеходными дорожками из new_paths_gdf.
    """
    for idx, row in new_paths_gdf.iterrows():
        geometry = row['geometry']
        coords = list(geometry.coords)
        for i in range(len(coords)-1):
            u = coords[i]
            v = coords[i+1]
            length = Point(u).distance(Point(v))
            if G.has_edge(u, v):
                if length < G[u][v]['weight']:
                    G[u][v]['weight'] = length
            else:
                G.add_edge(u, v, weight=length)
    print(f"Граф обновлён с {len(new_paths_gdf)} новыми дорожными сегментами.")
    return G

def calculate_intensity(houses_gdf):
    """
    Рассчитывает интенсивность движения пешеходов (N) для каждой группы населения.
    """
    # Определение количества жителей в доме
    if 'Apartments' not in houses_gdf.columns:
        print("Столбец 'Apartments' отсутствует в данных домов. Устанавливаем значение по умолчанию.")
        houses_gdf['Apartments'] = 1  # Значение по умолчанию

    houses_gdf['residents'] = houses_gdf['Apartments'] * 3  # количество квартир * коэффициент

    # Распределение по типам населения
    houses_gdf['children_pensioners'] = houses_gdf['residents'] * 0.15
    houses_gdf['adults_personal_transport'] = houses_gdf['residents'] * 0.30
    houses_gdf['adults_public_transport'] = houses_gdf['residents'] * 0.45
    houses_gdf['adults_cars'] = houses_gdf['residents'] * 0.10

    # Рассчитаем интенсивность движения пешеходов (N)
    # Увеличенный коэффициент для достижения более высоких значений
    houses_gdf.loc[:, 'intensity_children_pensioners'] = houses_gdf['children_pensioners'] * 1.0  # Коэффициент интенсивности
    houses_gdf.loc[:, 'intensity_adults_public_transport'] = houses_gdf['adults_public_transport'] * 0.5  # Коэффициент интенсивности

    # Общая интенсивность
    houses_gdf.loc[:, 'total_intensity'] = houses_gdf['intensity_children_pensioners'] + houses_gdf['intensity_adults_public_transport']

    return houses_gdf

years = 1

print(f"\n--- Обработка данных для {years} года ---")

# Определение целевой системы координат
target_crs = "EPSG:3857"  # Замените на вашу целевую CRS, если необходимо

# Шаг 1: Загрузка данных улиц
streets_files = [f"Streets_{i}очередь.shp" for i in range(1,4)] + ["Streets_исходные.shp"]
streets_gdfs = []
for file in streets_files:
    path = os.path.join(data_dir, file)
    gdf = load_and_reproject(path, target_crs)
    if gdf is not None:
        streets_gdfs.append(gdf)
    else:
        print(f"Файл {file} не был загружен и будет пропущен.")
if streets_gdfs:
    streets_combined = pd.concat(streets_gdfs, ignore_index=True)
else:
    streets_combined = gpd.GeoDataFrame(columns=['geometry'], crs=target_crs)
streets_combined = clean_geometries(streets_combined)
streets_combined


--- Обработка данных для 1 года ---
Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\Streets_1очередь.shp загружен успешно с 27521 записями.
Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\Streets_2очередь.shp загружен успешно с 28069 записями.
Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\Streets_3очередь.shp загружен успешно с 28817 записями.
Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\Streets_исходные.shp загружен успешно с 27209 записями.
Количество некорректных геометрий после исправления: 0
Все геометрии корректны.


,ST_NAME,ST_TYP_BEF,ST_NM_BASE,ROAD_CATEG,RoadDirect,RbndStght,RbndBck,Width,MaxSpdDrct,AvgSpdDrct,...,IsFerry,Style,U_TURN,OriginId,TrackNames,TopoNames1,duplicat,payment,material,length
0,None,None,None,Прочие улицы города,T,1.0,1.0,6.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Проектируемый 7048-й проезд,проезд,Проектируемый 7048-й,Магистральные улицы города,Any,2.0,2.0,12.0,60,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,None,None,Прочие улицы города,F,1.0,1.0,6.0,40,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,None,None,Магистральные улицы города,F,2.0,1.0,9.0,60,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,None,None,Прочие улицы города,T,1.0,1.0,6.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111611,None,None,None,Внутриквартальные проезды,Any,1.0,1.0,6.0,20,20,...,F,0.0,3.0,70030076977813673,None,None,None,None,None,12.246
111612,None,None,None,Внутриквартальные проезды,Any,1.0,1.0,6.0,20,20,...,F,0.0,3.0,70030076977813674,None,None,None,None,None,41.804
111613,None,None,None,Внутриквартальные проезды,Any,1.0,1.0,6.0,20,20,...,F,0.0,3.0,70030076977816202,None,None,None,None,None,22.386
111614,None,None,None,Внутриквартальные проезды,Any,1.0,1.0,6.0,20,20,...,F,0.0,3.0,70030076977816224,None,None,None,None,None,251.115


In [35]:
# Шаг 2: Загрузка зданий (существующих и планируемых)
buildings_files = ["Дома_исходные.shp"] + [f"House_{i}очередь_ЖК.shp" for i in range(1,4)]
buildings_gdfs = []
for file in buildings_files:
    path = os.path.join(data_dir, file)
    gdf = load_and_reproject(path, target_crs)
    if gdf is not None:
        buildings_gdfs.append(gdf)
    else:
        print(f"Файл {file} не был загружен и будет пропущен.")
if buildings_gdfs:
    buildings_combined = pd.concat(buildings_gdfs, ignore_index=True)
else:
    buildings_combined = gpd.GeoDataFrame(columns=['geometry'], crs=target_crs)
buildings_combined = clean_geometries(buildings_combined)
buildings_combined

C:\Users\hedge\DataspellProjects\vklab_recsys\venv\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\Дома_исходные.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\Дома_исходные.shp загружен успешно с 16386 записями.
Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\House_1очередь_ЖК.shp загружен успешно с 45 записями.
Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\House_2очередь_ЖК.shp загружен успешно с 126 записями.
Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\House_3очередь_ЖК.shp загружен успешно с 3 записями.
Количество некорректных геометрий после исправления: 595
Некорректные геометрии найдены:
     Name        Type    Purpose  \
2    None  Жилые дома  Жилой дом   
203  None  Жилые дома  Жилой дом   
204  None  Жилые дома  Жилой дом   
205  None  Жилые дома  Жилой дом   
207  None  Жилые дома  Жилой дом   

                                              geometry  
2    MULTIPOLYGON (((4168473.52 7465603.51, 4168486...  
203  MULTIPOLYGON (((4171456.38 7483279.58, 4171437...  
204  MULTIPOLYGON (((4171051

,Name,Type,Caption,Purpose,PostIndex,Elevation,Entrances,Apartments,District,DistrictId,...,StreetId2,Number2,Street3,StreetId3,Number3,Street4,StreetId4,Number4,Material,geometry
0,None,Жилые дома,None,Таунхаус,108803,2.0,NaN,NaN,Коммунарка,4504209520926899,...,None,None,None,None,None,None,None,None,Железобетон,"POLYGON ((4165306.35 7463920.69, 4165286.05 74..."
1,None,Жилые дома,None,Жилой дом,108803,10.0,3.0,108.0,Коммунарка,4504209520926899,...,None,None,None,None,None,None,None,None,Панель,"POLYGON ((4167970.75 7465703.44, 4167992.17 74..."
2,None,Жилые дома,None,Жилой дом,108803,9.0,2.0,67.0,Коммунарка,4504209520926899,...,None,None,None,None,None,None,None,None,Кирпич,"POLYGON ((4168465.81 7465519.85, 4168452.65 74..."
3,None,Жилые дома,None,Малоэтажный жилой дом,108803,3.0,3.0,18.0,Коммунарка,4504209520926899,...,None,None,None,None,None,None,None,None,Кирпич,"POLYGON ((4168258.38 7464679.75, 4168150.51 74..."
4,None,Жилые дома,None,Малоэтажный жилой дом,108803,3.0,2.0,24.0,Коммунарка,4504209520926899,...,None,None,None,None,None,None,None,None,Кирпич,"POLYGON ((4168562.63 7464563.77, 4168503.49 74..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16555,NaN,Известный по назначению,NaN,Хозяйственный корпус,NaN,1.0,NaN,NaN,Коммунарка,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((4173009.69 7469303.66, 4173011.59 74..."
16556,NaN,Известный по назначению,NaN,Автостоянка,NaN,1.0,NaN,NaN,Коммунарка,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((4171822.07 7468148.07, 4171828.11 74..."
16557,NaN,Школы,NaN,Школа,NaN,2.0,NaN,NaN,Коммунарка,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((4173932.77 7468595.71, 4173947.46 74..."
16558,NaN,Школы,NaN,Школа,NaN,3.0,NaN,NaN,Коммунарка,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((4173156.96 7467805.91, 4173164.36 74..."


In [46]:
# Шаг 3: Построение графа дорожной сети
G = build_graph(streets_combined.head(3000))

C:\Users\hedge\AppData\Local\Temp\ipykernel_22576\1201323181.py:134: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geometry.type == 'LineString':


In [47]:
# Шаг 4: Обработка связанных компонентов
G_largest = get_largest_component(G)
G_largest

Количество связанных компонентов в графе: 754
Компонент 1: 236 узлов
Компонент 2: 220 узлов
Компонент 3: 115 узлов
Компонент 4: 111 узлов
Компонент 5: 101 узлов
Компонент 6: 85 узлов
Компонент 7: 78 узлов
Компонент 8: 76 узлов
Компонент 9: 76 узлов
Компонент 10: 69 узлов
Компонент 11: 67 узлов
Компонент 12: 63 узлов
Компонент 13: 56 узлов
Компонент 14: 54 узлов
Компонент 15: 53 узлов
Компонент 16: 51 узлов
Компонент 17: 47 узлов
Компонент 18: 46 узлов
Компонент 19: 46 узлов
Компонент 20: 45 узлов
Компонент 21: 44 узлов
Компонент 22: 44 узлов
Компонент 23: 44 узлов
Компонент 24: 43 узлов
Компонент 25: 43 узлов
Компонент 26: 42 узлов
Компонент 27: 42 узлов
Компонент 28: 41 узлов
Компонент 29: 39 узлов
Компонент 30: 39 узлов
Компонент 31: 38 узлов
Компонент 32: 38 узлов
Компонент 33: 37 узлов
Компонент 34: 37 узлов
Компонент 35: 36 узлов
Компонент 36: 36 узлов
Компонент 37: 35 узлов
Компонент 38: 35 узлов
Компонент 39: 34 узлов
Компонент 40: 33 узлов
Компонент 41: 33 узлов
Компонент 42: 3

In [ ]:
# Визуализация графа
plt.figure(figsize=(20, 20))
pos = nx.spring_layout(G_largest)
nx.draw(G_largest, pos, with_labels=True, node_color='lightblue',
        font_size=10, font_weight='bold',
        connectionstyle='arc3,rad=0.3')
plt.title('Road Network Graph (First 500 Rows)')
plt.axis('off')
plt.show()

In [49]:
# Шаг 5: Загрузка и очистка данных домов и ключевых точек
houses_orig = load_and_reproject(os.path.join(data_dir, "Дома_исходные.shp"), target_crs)
houses_orig = clean_geometries(houses_orig)

C:\Users\hedge\DataspellProjects\vklab_recsys\venv\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\Дома_исходные.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\Дома_исходные.shp загружен успешно с 16386 записями.
Количество некорректных геометрий после исправления: 527
Некорректные геометрии найдены:
     Name        Type    Purpose  \
2    None  Жилые дома  Жилой дом   
203  None  Жилые дома  Жилой дом   
204  None  Жилые дома  Жилой дом   
205  None  Жилые дома  Жилой дом   
207  None  Жилые дома  Жилой дом   

                                              geometry  
2    MULTIPOLYGON (((4168473.52 7465603.51, 4168486...  
203  MULTIPOLYGON (((4171456.38 7483279.58, 4171437...  
204  MULTIPOLYGON (((4171051.98 7483211.1, 4171201....  
205  MULTIPOLYGON (((4171529.31 7483463.04, 4171519...  
207  MULTIPOLYGON (((4171663.79 7483309.12, 4171643...  
Количество некорректных геометрий после дополнительного исправления: 0
Все геометрии корректны после дополнительного исправления.


In [50]:
# Загрузка ключевых точек: Выходы метро и Остановки ОТ
metro_exits = load_and_reproject(os.path.join(data_dir, "Выходы_метро.shp"), target_crs)
bus_stops = load_and_reproject(os.path.join(data_dir, "Остановки_ОТ.shp"), target_crs)
houses3 = load_and_reproject(os.path.join(data_dir, "House_3очередь_ЖК.shp"), target_crs)

Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\Выходы_метро.shp загружен успешно с 7 записями.
Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\Остановки_ОТ.shp загружен успешно с 61 записями.
Файл C:\Users\hedge\DataspellProjects\cp_2024_graphs\Векторные данные\House_3очередь_ЖК.shp загружен успешно с 3 записями.


In [51]:
# Фильтрация школ из House_3очередь_ЖК.shp
if 'Type' in houses3.columns:
    schools = houses3[houses3['Type'] == 'Школы']
else:
    print("Столбец 'Type' отсутствует в данных House_3очередь_ЖК.shp. Фильтрация школ невозможна.")
    schools = gpd.GeoDataFrame(columns=houses3.columns, crs=target_crs)

In [56]:
# Объединение ключевых точек
key_points = pd.concat([metro_exits, bus_stops, schools], ignore_index=True)
key_points = clean_geometries(key_points)
key_points.head()

Количество некорректных геометрий после исправления: 1
Некорректные геометрии найдены:
   Name   Type Purpose                                           geometry
69  NaN  Школы   Школа  MULTIPOLYGON (((4173174.84 7467838.91, 4173121...
Количество некорректных геометрий после дополнительного исправления: 0
Все геометрии корректны после дополнительного исправления.


,Number,Text,geometry,TrType,Name,TrStopId,Type,Purpose,Elevation,Entrances,Apartments,District,Street
0,7,Потапово,POLYGON EMPTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,Потапово,POLYGON EMPTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,Потапово,POLYGON EMPTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Потапово,POLYGON EMPTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Потапово,POLYGON EMPTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Шаг 6: Привязка домов и ключевых точек к графу
houses_orig = bind_objects_to_graph(G_largest, houses_orig, object_type="Дом")
key_points = bind_objects_to_graph(G_largest, key_points, object_type="Ключевая точка")


Количество Дом(ов) без привязки к узлам графа: 0
Все Дом(ы) успешно привязаны к ближайшим узлам графа.
Удалено 68 Ключевая точка(ов) с пустой геометрией.

Количество Ключевая точка(ов) без привязки к узлам графа: 0
Все Ключевая точка(ы) успешно привязаны к ближайшим узлам графа.


In [31]:
# Шаг 7: Расчёт интенсивности пешеходов
houses_orig = calculate_intensity(houses_orig)

# Устранение предупреждения SettingWithCopyWarning с использованием .copy()
houses_orig = houses_orig.copy()

# Дополнительный вывод для проверки
print("Статистика по интенсивности:")
try:
    print(houses_orig[['intensity_children_pensioners', 'intensity_adults_public_transport', 'total_intensity']].describe())
except KeyError as e:
    print(f"Отсутствует столбец: {e}. Проверьте правильность расчёта интенсивности.")

# --- Пример Основного Процесса для Множественных Годов ---

Статистика по интенсивности:
       intensity_children_pensioners  intensity_adults_public_transport  \
count                     511.000000                         511.000000   
mean                       92.639237                         138.958855   
std                       112.249623                         168.374435   
min                         0.450000                           0.675000   
25%                        22.050000                          33.075000   
50%                        57.150000                          85.725000   
75%                       116.550000                         174.825000   
max                       697.950000                        1046.925000   

       total_intensity  
count       511.000000  
mean        231.598092  
std         280.624058  
min           1.125000  
25%          55.125000  
50%         142.875000  
75%         291.375000  
max        1744.875000  
